In [102]:
import pandas as pd

In [103]:
station_obs_names_df = pd.read_csv(r"E:\GitHub\Resources\1-TDM\Station-Lookups\station-obs-names.csv")
crt_station_obs_names_df = station_obs_names_df[station_obs_names_df['mode']=='CRT']
crt_station_obs_names_df = crt_station_obs_names_df.groupby(['LAT','LON'],as_index=False).agg(STATION=('STATION','first'))


# Sort by LAT (descending) and LON (ascending) for north-to-south ordering
unique_stations_df = crt_station_obs_names_df.drop_duplicates(subset=['STATION']).sort_values(
    by=['LAT', 'LON'], 
    ascending=[False, False]
).reset_index(drop=True)

# Assign row numbers only to unique STATIONs
unique_stations_df['RowNumber'] = range(1, len(unique_stations_df) + 1)

# Merge back to original DataFrame to retain all rows
crt_station_obs_names_df = crt_station_obs_names_df.merge(
    unique_stations_df[['STATION', 'RowNumber']], 
    on='STATION', 
    how='left'
)

crt_station_obs_names_df['STATION_ID'] = crt_station_obs_names_df['RowNumber'].apply(lambda x: f"{x:02d}") + '-' + crt_station_obs_names_df['STATION']

crt_station_obs_names_df

,LAT,LON,STATION,RowNumber,STATION_ID
0,40.225440,-111.660632,Provo Central,15,15-Provo Central
1,40.280140,-111.725489,Orem Central,14,14-Orem Central
2,40.374774,-111.820649,American Fork,13,13-American Fork
3,40.425196,-111.896354,Lehi,12,12-Lehi
4,40.515484,-111.904407,Draper,11,11-Draper
5,40.563155,-111.900753,South Jordan,10,10-South Jordan
6,40.659631,-111.895661,Murray Central,9,09-Murray Central
7,40.659758,-111.896432,Murray Central,9,09-Murray Central
8,40.660269,-111.895282,Murray Central,9,09-Murray Central
9,40.761721,-111.908301,Salt Lake Central,8,08-Salt Lake Central


In [113]:
obs_df = pd.read_csv(r"C:\Users\bhereth\Documents\2019 Final Weighted UTA OD Data - 2022-04-05 - processed.csv", low_memory=False)
obs_crt_df = obs_df[(obs_df['Linked_Mode_txt']=='CRT') & (obs_df['Ac_Mode_Model']=='Drive')]

# all HBC are peak period
obs_crt_df.loc[obs_crt_df['Purp5_text'] == 'HBC', 'PK_OK'] = 'PK'

obs_crt_df

,id,date_completed,final_route_surveyed_code,final_route_surveyed,direction,mode,purpose,resident_or_visitor_code,home_or_hotel_address,home_or_hotel_city,...,FirstOn_lat,FirstOn_lon,LastOff_lat,LastOff_lon,p_Stop_lat,p_Stop_lon,a_Stop_lat,a_Stop_lon,unlinked_weight_adj,linked_weight_adj
3,30.0,9-Sep-19,SLC_1_703_00,TRAX RED LINE TO DAYBREAK PARKWAY STATION,0.0,1.0,2.0,1.0,262 E Whitlock Ave,South Salt Lake,...,40.724091,-111.896892,40.280140,-111.725489,40.724091,-111.896892,40.280140,-111.725489,11.440,5.720
5,32.0,9-Sep-19,SLC_1_701_00,TRAX BLUE LINE TO DRAPER TOWN CENTER STATION,0.0,1.0,1.0,1.0,10404 Avondale Dr,Cedar Hills,...,40.374774,-111.820649,40.767452,-111.891101,40.374774,-111.820649,40.767452,-111.891101,18.136,9.068
46,212.0,11-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,1.0,1.0,W Coyote Cir,Saratoga Springs,...,40.425196,-111.896354,40.763855,-111.891071,40.425196,-111.896354,40.763855,-111.891071,9.312,4.656
100,331.0,11-Sep-19,SLC_1_703_01,TRAX RED LINE TO UNIVERSITY MEDICAL CENTER,1.0,1.0,2.0,1.0,4424 Vermillion Dr S,South Jordan,...,40.559038,-112.024643,40.245802,-111.644618,40.559038,-112.024643,40.245802,-111.644618,5.515,1.379
105,338.0,11-Sep-19,SLC_1_703_01,TRAX RED LINE TO UNIVERSITY MEDICAL CENTER,1.0,1.0,2.0,1.0,N Freedom Blvd & Bulldog Blvd,Provo,...,40.225440,-111.660632,40.760509,-111.839146,40.225440,-111.660632,40.760509,-111.839146,3.395,1.697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13402,22721.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,350 22nd St,Ogden,...,40.772532,-111.905124,41.224285,-111.980631,41.224285,-111.980631,40.772532,-111.905124,11.951,11.951
13403,22722.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,641 1850 N St,Ogden,...,40.763855,-111.891071,41.224285,-111.980631,41.224285,-111.980631,40.763855,-111.891071,11.951,5.976
13409,22737.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,185 E 1050 N,Bountiful,...,40.768217,-111.891081,40.880457,-111.903151,40.880457,-111.903151,40.768217,-111.891081,17.630,8.815
13410,22738.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,2803 W 1475 N,Layton,...,40.768217,-111.891081,41.094769,-112.013807,41.094769,-112.013807,40.768217,-111.891081,11.951,5.976


In [105]:
# check total should be around 20,500
obs_crt_df['linked_weight_adj'].sum()

20482.226000000002

In [106]:
stop_fields_lat_lon_df = pd.DataFrame([
    ['stop_on_lat'            , 'stop_on_long'              ],
    ['stop_off_lat'           , 'stop_off_long'             ],
    ['prev_tran_1_on_bus_lat' , 'prev_tran_1_on_bus_long'   ],
    ['prev_tran_1_off_bus_lat', 'prev_tran_1_off_bus_long'  ],
    ['prev_tran_2_on_bus_lat' , 'prev_tran_2_on_bus_long'   ],
    ['prev_tran_2_off_bus_lat', 'prev_tran_2_off_bus_long'  ],
    ['prev_tran_3_on_bus_lat' , 'prev_tran_3_on_bus_long'   ],
    ['prev_tran_3_off_bus_lat', 'prev_tran_3_off_bus_long'  ],
    ['prev_tran_4_on_bus_lat' , 'prev_tran_4_on_bus_long'   ],
    ['prev_tran_4_off_bus_lat', 'prev_tran_4_off_bus_long'  ],
    ['next_tran_1_on_bus_lat' , 'next_tran_1_on_bus_long'   ],
    ['next_tran_1_off_bus_lat', 'next_tran_1_off_bus_long'  ],
    ['next_tran_2_on_bus_lat' , 'next_tran_2_on_bus_long'   ],
    ['next_tran_2_off_bus_lat', 'next_tran_2_off_bus_long'  ],
    ['next_tran_3_on_bus_lat' , 'next_tran_3_on_bus_long'   ],
    ['next_tran_3_off_bus_lat', 'next_tran_3_off_bus_long'  ],
    ['next_tran_4_on_bus_lat' , 'next_tran_4_on_bus_long'   ],
    ['next_tran_4_off_bus_lat', 'next_tran_4_off_bus_long'  ]
], columns=['stop_lat','stop_lon'])
stop_fields_lat_lon_df

,stop_lat,stop_lon
0,stop_on_lat,stop_on_long
1,stop_off_lat,stop_off_long
2,prev_tran_1_on_bus_lat,prev_tran_1_on_bus_long
3,prev_tran_1_off_bus_lat,prev_tran_1_off_bus_long
4,prev_tran_2_on_bus_lat,prev_tran_2_on_bus_long
5,prev_tran_2_off_bus_lat,prev_tran_2_off_bus_long
6,prev_tran_3_on_bus_lat,prev_tran_3_on_bus_long
7,prev_tran_3_off_bus_lat,prev_tran_3_off_bus_long
8,prev_tran_4_on_bus_lat,prev_tran_4_on_bus_long
9,prev_tran_4_off_bus_lat,prev_tran_4_off_bus_long


In [107]:
obs_crt_df.columns.to_list()

['id',
 'date_completed',
 'final_route_surveyed_code',
 'final_route_surveyed',
 'direction',
 'mode',
 'purpose',
 'resident_or_visitor_code',
 'home_or_hotel_address',
 'home_or_hotel_city',
 'home_or_hotel_state',
 'home_or_hotel_zip',
 'home_or_hotel_lat',
 'home_or_hotel_long',
 'final_origin_place_type_code',
 'origin_address',
 'origin_city',
 'origin_state',
 'origin_zip',
 'origin_lat',
 'origin_long',
 'prev_transfers_code',
 'trip_first_route_code',
 'trip_first_route',
 'trip_first_route_other',
 'trip_second_route_code',
 'trip_second_route',
 'trip_second_route_other',
 'trip_third_route_code',
 'trip_third_route',
 'trip_third_route_other',
 'trip_fourth_route_code',
 'trip_fourth_route',
 'trip_fourth_route_other',
 'final_origin_transport_code',
 'final_destin_place_type_code',
 'destin_address',
 'destin_city',
 'destin_state',
 'destin_zip',
 'destin_lat',
 'destin_long',
 'next_transfers_code',
 'trip_next_route_code',
 'trip_next_route',
 'trip_next_route_other',


In [108]:
filtered_obs_fields = ['id','Purp5_text','PK_OK','Ac_Mode_Model','Linked_Mode_txt','linked_weight_adj']
filtered_obs_fields

['id',
 'Purp5_text',
 'PK_OK',
 'Ac_Mode_Model',
 'Linked_Mode_txt',
 'linked_weight_adj']

In [109]:
obs_crt_stations_df = pd.DataFrame()

for _, row in stop_fields_lat_lon_df.iterrows():
    #print(_)

    # get station name
    _df = pd.merge(obs_crt_df, crt_station_obs_names_df, left_on=[row['stop_lat'], row['stop_lon']], right_on=['LAT','LON'])

    _df = _df[filtered_obs_fields + ['STATION_ID']]

    obs_crt_stations_df = pd.concat([obs_crt_stations_df, _df])

display(obs_crt_stations_df)

,id,Purp5_text,PK_OK,Ac_Mode_Model,Linked_Mode_txt,linked_weight_adj,STATION_ID
0,32.0,HBW,PK,Drive,CRT,9.068,08-Salt Lake Central
1,61.0,NHB,OK,Walk,CRT,1.633,09-Murray Central
2,338.0,HBC,PK,Drive,CRT,1.697,09-Murray Central
3,430.0,HBO,OK,Walk,CRT,1.573,07-North Temple
4,434.0,HBW,OK,Drive,CRT,2.360,07-North Temple
...,...,...,...,...,...,...,...
29,18561.0,HBW,OK,Walk,CRT,6.300,11-Draper
30,19897.0,HBW,PK,Drive,CRT,5.046,02-Roy
31,21157.0,NHB,PK,Walk,CRT,7.784,05-Farmington
0,11872.0,HBW,OK,Drive,CRT,2.589,07-North Temple


In [110]:
obs_crt_stations_unique_df = obs_crt_stations_df.drop_duplicates()

grouped_df = obs_crt_stations_unique_df.groupby(
    ['id', 'linked_weight_adj', 'Purp5_text', 'PK_OK', 'Ac_Mode_Model', 'Linked_Mode_txt'],
    as_index=False
).agg({'STATION_ID': lambda x: sorted(x)})

# Extract STATION_ID_1 and STATION_ID_2 from the list
grouped_df['STATION_ID_1'] = grouped_df['STATION_ID'].apply(lambda x: x[0] if len(x) > 0 else None)
grouped_df['STATION_ID_2'] = grouped_df['STATION_ID'].apply(lambda x: x[1] if len(x) > 1 else None)

# Drop the original list column to keep only the new columns
grouped_df = grouped_df.drop(columns=['STATION_ID'])

grouped_again_df = grouped_df.groupby(['Purp5_text','PK_OK','Ac_Mode_Model','STATION_ID_1','STATION_ID_2'], as_index=False).agg(trips=('linked_weight_adj','sum'))
grouped_again_df['trips'] = grouped_again_df['trips'].round(2)

grouped_again_df['PurpPrdAcMode'] = grouped_again_df['Purp5_text'] + '_' + grouped_again_df['PK_OK'] + "_" + grouped_again_df['Ac_Mode_Model']
grouped_again_df.drop(columns=['Purp5_text','PK_OK','Ac_Mode_Model'], inplace=True)
pivot_df = grouped_again_df.pivot(index=['STATION_ID_1','STATION_ID_2'], columns='PurpPrdAcMode', values='trips').fillna(0).reset_index()
pivot_df

PurpPrdAcMode,STATION_ID_1,STATION_ID_2,HBC_PK_Drive,HBC_PK_Walk,HBO_OK_Drive,HBO_OK_Walk,HBO_PK_Drive,HBO_PK_Walk,HBSch_OK_Drive,HBSch_OK_Walk,HBSch_PK_Drive,HBSch_PK_Walk,HBW_OK_Drive,HBW_OK_Walk,HBW_PK_Drive,HBW_PK_Walk,NHB_OK_Drive,NHB_OK_Walk,NHB_PK_Drive,NHB_PK_Walk
0,01-Ogden,02-Roy,11.50,0.00,0.00,0.00,0.00,21.36,0.0,0.0,0.00,0.0,0.00,21.67,4.02,6.67,7.00,0.00,0.00,0.00
1,01-Ogden,03-Clearfield,132.14,2.96,36.93,4.67,5.79,7.12,0.0,0.0,0.00,0.0,23.00,24.45,11.58,31.62,0.00,15.26,0.00,0.00
2,01-Ogden,04-Layton,23.43,4.74,2.46,1.87,0.00,0.00,0.0,0.0,0.00,0.0,0.00,8.75,37.10,14.72,1.23,8.93,0.00,0.00
3,01-Ogden,05-Farmington,11.44,1.23,22.51,19.06,36.65,0.00,0.0,0.0,0.00,2.9,2.46,26.68,28.58,3.61,0.00,5.61,0.00,10.68
4,01-Ogden,06-Woods Cross,33.82,2.80,10.88,7.91,0.00,0.00,0.0,0.0,3.36,0.0,3.70,10.02,13.94,3.36,8.82,0.00,7.22,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,12-Lehi,14-Orem Central,28.13,8.68,0.00,0.00,8.53,3.85,0.0,0.0,0.00,0.0,14.27,15.95,45.35,12.84,0.00,0.00,0.00,0.00
101,12-Lehi,15-Provo Central,29.13,2.51,27.51,19.96,5.92,3.88,0.0,0.0,0.00,0.0,58.80,12.90,228.41,34.68,8.84,16.10,7.70,0.00
102,13-American Fork,14-Orem Central,91.14,22.65,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,86.25,30.36,0.00,0.00,0.00,0.00
103,13-American Fork,15-Provo Central,187.52,0.00,23.29,2.03,12.05,0.00,0.0,0.0,0.00,0.0,11.38,0.00,125.24,0.00,30.98,0.00,0.00,0.00


In [111]:
# export station-to-staion pairs od
pivot_df.to_csv('station-pair-trips/station-pair-trips.csv', index=False)

In [112]:
# export station geojson

import geopandas as gpd

# Convert DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    crt_station_obs_names_df, 
    geometry=gpd.points_from_xy(crt_station_obs_names_df['LON'], crt_station_obs_names_df['LAT'])
)

# Define CRS (Coordinate Reference System) - Assuming WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Export to GeoJSON
geojson_path = "station-pair-trips/stations.geojson"  # Path for the exported file
gdf.to_file(geojson_path, driver="GeoJSON")

# Export to Shapefile
shapefile_path = "station-pair-trips/stations.shp"
gdf.to_file(shapefile_path, driver="ESRI Shapefile")

print(f"GeoJSON exported to {geojson_path}")
print(f"Shapefile exported to {shapefile_path}")


GeoJSON exported to station-pair-trips/stations.geojson
Shapefile exported to station-pair-trips/stations.shp
